In [10]:
from pyspark.sql import SparkSession
spark =(
    SparkSession.builder.appName("Spark_SQL").getOrCreate()
)

In [11]:
csvFile="Data/departuredelays.csv"
df = (
    spark.read.format("csv")
    .option("inferSchema", "true")
    .option("header", "true")
    .load(csvFile)
)
df.createOrReplaceTempView("us_delay_flights_tbl")

In [12]:
# If you want to specify a schema, you can use a DDL-formatted string. For example:
schema = "`date` STRING, `delay` INT, `distance` INT,`origin` STRING, `destination` STRING"


In [13]:
spark.sql(
    """
    select distance, origin, destination
    from us_delay_flights_tbl
    where distance > 1000
    order by distance desc
    """
).show(10)


+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



In [14]:
spark.sql(
    """
    select date, delay, origin, destination
    from us_delay_flights_tbl
    where delay > 120 and origin = 'SFO' and destination = 'ORD'
    order by delay desc
    """
).show(10)

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|2190925| 1638|   SFO|        ORD|
|1031755|  396|   SFO|        ORD|
|1022330|  326|   SFO|        ORD|
|1051205|  320|   SFO|        ORD|
|1190925|  297|   SFO|        ORD|
|2171115|  296|   SFO|        ORD|
|1071040|  279|   SFO|        ORD|
|1051550|  274|   SFO|        ORD|
|3120730|  266|   SFO|        ORD|
|1261104|  258|   SFO|        ORD|
+-------+-----+------+-----------+
only showing top 10 rows



In [15]:
spark.sql(
    """
    select delay, origin, destination, \
    case \
        when delay > 360 then 'Very Long Delays' \
        when delay >= 120 and delay <= 360 then 'long Delays' \
        when delay >= 60 and delay < 120 then 'Short Delays' \
        when delay > 0 and delay < 60 then 'Toleable Delays' \
        when delay = 0 then 'No delays'
        else 'Early'
    end as flight_delays
    from us_delay_flights_tbl
    order by origin, delay desc
    """
).show(10)


+-----+------+-----------+-------------+
|delay|origin|destination|flight_delays|
+-----+------+-----------+-------------+
|  333|   ABE|        ATL|  long Delays|
|  305|   ABE|        ATL|  long Delays|
|  275|   ABE|        ATL|  long Delays|
|  257|   ABE|        ATL|  long Delays|
|  247|   ABE|        DTW|  long Delays|
|  247|   ABE|        ATL|  long Delays|
|  219|   ABE|        ORD|  long Delays|
|  211|   ABE|        ATL|  long Delays|
|  197|   ABE|        DTW|  long Delays|
|  192|   ABE|        ORD|  long Delays|
+-----+------+-----------+-------------+
only showing top 10 rows



In [16]:
from pyspark.sql.functions import col,desc
(
    df.select("distance", "origin", "destination")
    .where(col("distance") > 1000)
    .orderBy(desc("distance"))).show(10)
(
    df.select("distance", "origin", "destination")
    .where("distance > 1000")
    .orderBy("distance", ascending = False).show(10)
)
# spark.stop()

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



In [26]:
# Spark SQL interface to query data is similar to writing a regular SQL query to a relational database table.
# Spark manages all the complexities of creating and managing views and tables, both in memory and on disk.


In [27]:
# SQL Tables and Views
# Tables hold data. Associated with each table in Spark is its relevant metadata, which is information 
# about the table and its data: the schema, description, table name, database name, column names, partitions, 
# physical location where the actual data resides, etc. 


# Spark allows you to create two types of tables: managed and unmanaged. 
# For a managed table, Spark manages both the metadata and the data in the file store as Amazon S3 or Azure Blob


# For an unmanaged table, Spark only manages the metadata, while you manage the data yourself in an 
# external data source such as Cassandra.


# With a managed table, because Spark manages everything, a SQL command such as 
# DROP TABLE table_name deletes both the metadata and the data.

# With an unmanaged table, the same command will delete only the metadata, not the actual data


In [17]:
# Create database from spark sql
spark.sql("CREATE DATABASE Spark_SQl")


DataFrame[]

In [18]:
spark.sql("USE Spark_SQl")

DataFrame[]

In [19]:
# To create a managed table within the database learn_spark_db
schema = "`date` STRING, `delay` INT, `distance` INT,`origin` STRING, `destination` STRING"
flights_df = spark.read.csv(csvFile, schema=schema)
flights_df.write.saveAsTable("Delay")

In [20]:
# Creating an unmanaged table with API dataframe
schema = "`date` STRING, `delay` INT, `distance` INT,`origin` STRING, `destination` STRING"
flights_df = spark.read.csv(csvFile, schema=schema)
# flights_df.write.saveAsTable("Vynguyen")
(flights_df
  .write
  .option("path", "Data/us_delay_flights_tbl")
  .saveAsTable("us_delay_flights_tbl"))

In [23]:
# Creating Views
# Spark can create views on top of existing tables
# Views can be global (visible across all SparkSessions on a given cluster)
# Session-scoped (visible only to a single SparkSession)

#  The difference between a view and a table is that views don’t actually hold the data; 
# tables persist after your Spark application terminates, but views disappear.

df_sfo = spark.sql(
    "SELECT date, delay, origin, destination \
    from us_delay_flights_tbl where origin = 'SFO' \
    "
)
df_jfk = spark.sql("SELECT date, delay, origin, destination FROM \
  us_delay_flights_tbl WHERE origin = 'JFK'")

df_sfo.createOrReplaceGlobalTempView("us_origin_airport_SFO_global_tmp_view")
df_jfk.createOrReplaceTempView("us_origin_airport_JFK_tmp_view")
spark.read.table("us_origin_airport_JFK_tmp_view")
spark.sql("SELECT * FROM us_origin_airport_JFK_tmp_view")
# drop views
# spark.catalog.dropGlobalTempView("us_origin_airport_SFO_global_tmp_view")
# spark.catalog.dropTempView("us_origin_airport_JFK_tmp_view")

DataFrame[date: int, delay: int, origin: string, destination: string]

In [25]:
# Temporary views versus global temporary views
# The difference between temporary and global temporary views being subtle, 
# it can be a source of mild confusion among developers new to Spark.

# A temporary view is tied to a single SparkSession within a Spark application
# a global temporary view is visible across multiple SparkSessions within a Spark application

# Viewing the Metadata
# Spark manages the metadata associated with each managed or unmanaged table
# a high-level abstraction in Spark SQL for storing metadata.
spark.catalog.listDatabases()
spark.catalog.listTables()
spark.catalog.listColumns("us_delay_flights_tbl")

[Column(name='date', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='delay', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='distance', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='origin', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='destination', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False)]

In [26]:
# Caching SQL Tables
# Reading Tables into DataFrames
us_flights_df = spark.sql("SELECT * FROM us_delay_flights_tbl")
us_flights_df2 = spark.table("us_delay_flights_tbl")
us_flights_df2.show()


+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|    6|     602|   ABE|        ATL|
|1020600|   -8|     369|   ABE|        DTW|
|1021245|   -2|     602|   ABE|        ATL|
|1020605|   -4|     602|   ABE|        ATL|
|1031245|   -4|     602|   ABE|        ATL|
|1030605|    0|     602|   ABE|        ATL|
|1041243|   10|     602|   ABE|        ATL|
|1040605|   28|     602|   ABE|        ATL|
|1051245|   88|     602|   ABE|        ATL|
|1050605|    9|     602|   ABE|        ATL|
|1061215|   -6|     602|   ABE|        ATL|
|1061725|   69|     602|   ABE|        ATL|
|1061230|    0|     369|   ABE|        DTW|
|1060625|   -3|     602|   ABE|        ATL|
|1070600|    0|     369|   ABE|        DTW|
|1071725|    0|     602|   ABE|        ATL|
|1071230|    0|     369|   ABE|        DTW|
|1070625|    0|     602|   ABE|        ATL|
|1071219|    0|     569|   ABE|        ORD|
|1080600|    0|     369|   ABE| 

In [150]:
# Data Sources for DataFrames and SQL Tables
# available file formats, and ways to load and write data, along with specific 
# options pertaining to these data sources. 


In [152]:

# DataFrameReader
# DataFrameReader is the core construct for reading data from a data source into a DataFrame.
# It has a defined format and a recommended pattern for usage:

# DataFrameReader.format(args).option("key", "value").schema(args).load()

# SparkSession.read 
# // or 
# SparkSession.readStream


# Parquet is the default and preferred data source for Spark because it’s efficient, 
# uses columnar storage, and employs a fast compression algorithm. 
# when we cover the Catalyst optimizer in greater depth.



In [153]:
# DataFrameWriter
# DataFrameWriter does the reverse of its counterpart:  it saves or writes data to a specified built-in data source

# Format:
# DataFrameWriter.format(args)
#   .option(args)
#   .bucketBy(args)
#   .partitionBy(args)
#   .save(path)

# DataFrameWriter.format(args).option(args).sortBy(args).saveAsTable(table)

# DataFrame.write
# // or 
# DataFrame.writeStream

In [156]:
# Parquet
# Parquet is an open source columnar file format that offers many I/O optimizations 
# (such as compression, which saves storage space and allows for quick access to data columns).

# Because of its efficiency and these optimizations, we recommend that after you have transformed and 
# cleansed your data, you save your DataFrames in the Parquet format for downstream consumption

# Reading Parquet files into a DataFrame

# file = """/databricks-datasets/learning-spark-v2/flights/summary-data/parquet/\
#   2010-summary.parquet/"""
# df = spark.read.format("parquet").load(file)



In [157]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show(10)

+----+-----+--------+------+-----------+
|date|delay|distance|origin|destination|
+----+-----+--------+------+-----------+
+----+-----+--------+------+-----------+

